Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [22]:
%pip install openai --upgrade
%pip install pydub simpleaudio
%pip install nltk
%pip install pyaudio numpy
%pip install sounddevice scipy
%pip install -U openai-whisper

  Attempting uninstall: openai
    Found existing installation: openai 1.54.1
    Uninstalling openai-1.54.1:
      Successfully uninstalled openai-1.54.1
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/800.5 kB ? eta -:--:--
     ------------- -------------------------- 262.1/800.5 kB ? eta -:--:--
     -------------------------------------- 800.5/800.5 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 884.5/884.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803358 sha256=1933ef943b3b61efce9b28f5aab3f41399e4e3f3a99f2bb8a48982a56d402746
  Stored in directory: c:\users\mattj\appdata\local\pip\cache\wheels\2f\f2\ce\6eb23db4091d026238ce76703bd66da60b969d70bcc

In [16]:
# Get the openai secret key
import getpass
from openai import OpenAI

secret_key = getpass.getpass("Please enter your openai key: ")
import os
os.environ["OPENAI_API_KEY"]= secret_key


In [17]:

# OpenAI set up:
client = OpenAI()
client.api_key = secret_key
# Set up OpenAI client with the provided API key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
             {
                "role": "system",
                "content": "You are a real-time voice chat assistant for Spanish language learning, specializing in Mexican Spanish with a natural Mexican accent."
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

'¡Hola! Sí, aquí estoy. ¿Cómo te puedo ayudar con tu aprendizaje del español?'

In [3]:
'''
from pathlib import Path
from openai import OpenAI
#client = OpenAI()
filename1 = "speech.mp3"

speech_file_path = Path.cwd() / filename1
response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input="a tu orden"
)

response.stream_to_file(speech_file_path)
'''

C:\Users\mattj\AppData\Local\Temp\ipykernel_3080\3526707331.py:13: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [ ]:
from pydub import AudioSegment
import simpleaudio as sa
from pathlib import Path


# Play the intro.mp3 to begin things
filename1 = "intro.mp3"

#speech_file_path = Path("speech.mp3")
# Load and play the audio file
speech_file_path = Path.cwd() / filename1
audio = AudioSegment.from_mp3(speech_file_path)
play_obj = sa.play_buffer(
    audio.raw_data,
    num_channels=audio.channels,
    bytes_per_sample=audio.sample_width,
    sample_rate=audio.frame_rate
)
play_obj.wait_done()  # Wait until playback is finished


In [28]:
# Records user audio and detects pauses (silence) to segment the recording into separate chunks.
# When silence is detected for a specified duration, the audio is saved as an MP3 file. This allows for efficient 
# segmentation of audio data based on user pauses. The program also ensures that audio chunks do not exceed a set size.

import sounddevice as sd
import numpy as np
import os
import time
from pydub import AudioSegment
from scipy.io.wavfile import write

# Parameters
SAMPLE_RATE = 44100  # Sample rate in Hz
DURATION_THRESHOLD = 1.5  # Duration in seconds to consider silence
SILENCE_THRESHOLD = 0.01  # Amplitude threshold for silence
MAX_CHUNK_SIZE = 25 * 1024 * 1024  # Maximum chunk size in bytes (25 MB)
CHUNK_FOLDER = "audio_chunks"  # Folder to save audio chunks

# Create directory for audio chunks if it doesn't exist
if not os.path.exists(CHUNK_FOLDER):
    os.makedirs(CHUNK_FOLDER)

def is_silent(data):
    """Check if the audio data is silent by evaluating if its mean amplitude is below the silence threshold."""
    return np.abs(data).mean() < SILENCE_THRESHOLD

def record_audio():
    """Continuously record audio, saving each chunk when silence is detected for the threshold duration."""
    print("Recording... Speak now.")

    # Initialize the current chunk and silence detection timing
    current_chunk = []
    silence_start_time = None

    # Start the audio input stream
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype='float32') as stream:
        while True:
            # Read one second of audio data from the stream
            data = stream.read(SAMPLE_RATE)[0]
            current_chunk.extend(data.flatten())  # Add the new data to the current chunk

            # Check if the audio data is silent
            if is_silent(data):
                # If silence starts, begin a timer
                if silence_start_time is None:
                    silence_start_time = time.time()
            else:
                # Reset the timer if sound is detected
                silence_start_time = None

            # Stop recording if silence has lasted beyond the duration threshold
            if silence_start_time and (time.time() - silence_start_time) > DURATION_THRESHOLD:
                print("Silence detected. Stopping recording.")
                break  # Exit the loop to stop recording

    # Save the final chunk after breaking out of the loop
    if current_chunk:
        save_audio_chunk(current_chunk)
        print("Final chunk saved.")

def save_temp_wav(file_path, chunk):
    """Save the recorded audio chunk to a WAV file temporarily for further processing or conversion."""
    if len(chunk) == 0:
        return  # Don't save empty chunks
    
    # Convert to WAV and save
    audio_data = np.array(chunk, dtype=np.float32)
    write(file_path, SAMPLE_RATE, audio_data)

def convert_to_mp3(wav_file, mp3_file):
    """Convert a temporary WAV file to MP3 format using the Pydub library."""
    try:
        audio = AudioSegment.from_wav(wav_file)
        audio = audio.set_channels(1)  # Ensure mono channel
        audio = audio.set_frame_rate(16000)  # Set sample rate to 16 kHz (Whisper's recommended rate)
        audio.export(mp3_file, format="mp3", bitrate="128k")  # Set bitrate to 128 kbps
    except Exception as e:
        print(f"Error converting to MP3: {e}")


def save_audio_chunk(chunk):
    """Save the recorded audio chunk to an MP3 file, creating a unique file name."""
    if len(chunk) == 0:
        return  # Don't save empty chunks

    # Create a unique file name for the MP3 file
    chunk_index = len(os.listdir(CHUNK_FOLDER)) + 1
    mp3_file_path = os.path.join(CHUNK_FOLDER, f"chunk{chunk_index}.mp3")

    # Convert the chunk to WAV temporarily for saving
    temp_wav_path = os.path.join(CHUNK_FOLDER, "temp.wav")
    save_temp_wav(temp_wav_path, chunk)

    # Convert to MP3
    convert_to_mp3(temp_wav_path, mp3_file_path)

    print(f"Saved chunk to {mp3_file_path}")

if __name__ == "__main__":
    record_audio()


Recording... Speak now.
Silence detected. Stopping recording.
Saved chunk to audio_chunks\chunk9.mp3
Final chunk saved.


### Transcription

In [ ]:
import os
from openai import OpenAI

# Configuration
CHUNK_FOLDER = "audio_chunks"  # Directory where audio chunks are stored
USE_NEWEST_FILE = True         # Set to True to use the newest file, False to specify a file
SPECIFIED_FILE = ""            # Leave empty to use the newest file, or specify a filename like "speech.mp3"

# Initialize OpenAI client
client = OpenAI()

# Function to get the newest file from the directory
def get_newest_file(directory):
    """Return the newest file in the specified directory."""
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None
    files.sort(key=lambda f: os.path.getmtime(os.path.join(directory, f)), reverse=True)
    return os.path.join(directory, files[0])

# Determine which file to use
file_path = None
if USE_NEWEST_FILE:
    file_path = get_newest_file(CHUNK_FOLDER)
else:
    file_path = SPECIFIED_FILE if SPECIFIED_FILE else None

if file_path:
    # Open the audio file
    with open(file_path, "rb") as audio_file:
        # Perform transcription using Whisper model
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
        )

        # Print the transcription text
        print(transcription.text)
else:
    print("No audio file found. Please check the directory or specify a file.")


Yo ir al tienda para comprar manzanas y platanos. Yo fui al tienda para comprar manzanas y platanos.


In [ ]:
'''
#transcribe to text
audio_file = open("/path/to/file/speech.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(transcription.text)

'''

In [ ]:
#call gpt to check for grammar errors in the transcription

foundError = complete(f"{myPrompt2}\n\n{myText}")
print(foundError)


Yo ir al tienda para comprar manzanas y platanos. Yo FUI al tienda para comprar manzanas y platanos.


In [ ]:
# Divide up the transcription into sentences so each error can be looked at separately
import nltk
#nltk.download('punkt_tab')  # Download the punkt tokenizer if you haven't already

def split_paragraph_into_sentences(paragraph):
    sentences = nltk.tokenize.sent_tokenize(paragraph)
    return sentences

# Example usage
responseSentenceSplit = split_paragraph_into_sentences(foundError)
print(responseSentenceSplit)

['Yo ir al tienda para comprar manzanas y platanos.', 'Yo FUI al tienda para comprar manzanas y platanos.']


In [ ]:
#I think if we just said it this way to begin with then it will skip over all the logic that deals 
# withthe while loop and we can go straight to the user getting to talk right away 
# which would be very convenientand would reuse the code

errorsLeft = False

In [73]:
#convert errors into audio
if gptResponse = "No Errors.":
    errorsLeft = False
else:
    errorsLeft = True
    filename1 = "speech.mp3"
    speech_file_path = Path.cwd() / filename1
    response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=foundError
    )
    response.stream_to_file(speech_file_path)

while errorsLeft:   
    
    # play audio back
    audio = AudioSegment.from_mp3(speech_file_path)
    play_obj = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate
    )
    play_obj.wait_done()  # Wait until playback is finished

    # Record the user response
    # transcribe
    # Check if they got it right using a gpt call, if they did they change errorLeft
    # also check against other cases (as described below)
    errorsLeft = False

#No more errors, continue conversation, play 'cuentame.mp3' 
audio = AudioSegment.from_mp3(Path.cwd() / "cuentame.mp3")
    play_obj = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate
    )
    play_obj.wait_done()  # Wait until playback is finished

C:\Users\mattj\AppData\Local\Temp\ipykernel_3080\590219557.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [75]:

''' 
Listen to audio

if the last words said were  
nevermind - just move one. this is tough because i might just completely move on, so you need logic to notice this has happened and ignore checking against old errors
repite - auto replay everything that was just said and ignore 
repite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)

otherwise confirm that i repeated back the correction properly. 
'''

' \nif the last words said were  \nnevermind - just move one\nrepite - auto replay everything that was just said and ignore \nrepite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)\n\nListen to audio\nI respond normally - one by one, go through the things that were said to me and check if i said them correctly back. \n    if yes, then move to the next item,\n    if no, add that item back to the audio output\n    if all yes, then continue conversation\n    otherwise, concat audio again and play new audio\n\n'

In [14]:
# Deletes dead air at the beginning and end of MP3 files
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_silence(input_file, silence_thresh=-40, min_silence_len=300, buffer_ms=200):
    """
    Trim dead air (silence) from the beginning and end of an MP3 file, leaving a small buffer, and save it with '_trim' appended to the filename.
    
    :param input_file: Path to the input MP3 file
    :param silence_thresh: Silence threshold in dB (default is -40 dB)
    :param min_silence_len: Minimum length of silence to consider (in milliseconds)
    :param buffer_ms: Extra milliseconds to leave at the beginning and end of the trimmed audio (default is 100 ms)
    """
    # Load the audio file
    audio = AudioSegment.from_mp3(input_file)
    
    # Detect non-silent portions of the audio
    non_silent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    if non_silent_ranges:
        # Get the start and end of the non-silent range and add buffer
        start_trim = max(0, non_silent_ranges[0][0] - buffer_ms)
        end_trim = min(len(audio), non_silent_ranges[-1][1] + buffer_ms)
        
        # Trim the audio with the buffer
        trimmed_audio = audio[start_trim:end_trim]
        
        # Create output filename by appending '_trim' before the file extension
        base, ext = os.path.splitext(input_file)
        output_file = f"{base}_trim{ext}"
        
        # Export the trimmed audio to an MP3 file
        trimmed_audio.export(output_file, format="mp3")
        print(f"Trimmed audio saved as {output_file}")
    else:
        print("No non-silent segments detected. File not saved.")

# Example usage
input_mp3 = "./audio_chunks/chunk2.mp3"
trim_silence(input_mp3)


Trimmed audio saved as ./audio_chunks/chunk2_trim.mp3


In [ ]:
myText = "Yo ir al tienda para comprar manzanas y platanos. Cuando llegué, vi muchas frutas y verduras. El cajero me dijo que el precio son muy alto, pero no me importa. Después, yo regresar a casa y preparé un plato grande para mi familia. Todos nos gusta comer juntos."
#myText = "Hola, como estas"
#Prompt1 is show all the errors in a list
myPrompt1 = """
You will receive a paragraph written in Spanish. Your task is to identify and correct major grammatical errors while adhering to the following instructions:

1. Ignore gender errors.
2. If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
3. Ignore typing mistakes or missing accents; these are not considered errors.
4. Ignore minor clarity issues.
5. If there are no errors worth mentioning, output "No Errors."

Your output should consist only of the correct Spanish for each identified error, separated by periods. Do not provide any additional commentary or explanations. Aim for responses of 4-5 words unless more is needed.

Example:

Input: "Nosotros va a la playa mañana. Ellos trae sus juguetes."

Output: "Nosotros vamos. Ellos traen."
"""
 #prompt2 find one error and give the complete correction
myPrompt2 = """ 
You will receive a paragraph written in Spanish. Your task is to identify and correct one major grammatical error while adhering to the following instructions:
Ignore gender errors.
If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
Ignore typing mistakes or missing accents; these are not considered errors.
Ignore minor clarity issues.
If there are no significant errors, output "No Errors."
Your output should consist of three sentences:

The original incorrect sentence, abreviated to show what was said, but not include extraneous detail before and after in the sentence.
The corrected version with the corrected word in all caps.
If no significant error is found, just write "No Errors."
Example:

Input: Ayer, nosotros va a la playa mañana para solear. Ellos trae sus juguetes.

Output: Nosotros va a la playa mañana. Nosotros VAMOS a la playa mañana.
"""